# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

> You can use the classwork notebook and also [my Everything scraping reference](https://jonathansoma.com/everything/scraping)

## Setup: Import what you'll need to scrape the page

We'll be using either Playwrightfor this, *not* requests.

In [1]:
from playwright.async_api import async_playwright

In [16]:
import re
from bs4 import BeautifulSoup

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [10]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [11]:
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

<Response url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' request=<Request url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' method='GET'>>

In [12]:
await page.locator('input[id="pht_lnm"]').fill('Nguyen')

In [13]:
await page.locator('input[name="B1"]').click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't reaaallly tabular data (or there's just too much to clean up).

Once you've loaded the page: **what selector are you going to do use to find the data?** In class we used things like `.article` or `.head-list`.

> You honestly can do this all with Playwright, no BeautifulSoup involved! But we didn't cover that in class so you should stick to this method:
> 
> ```python
> html = await page.content()
> doc = BeautifulSoup(html)
> ```

In [17]:
html = await page.content()
doc = BeautifulSoup(html)

### Grab the rows, loop through each result and print the entire row

It's probably too many rows to do all at once: once you have a list, use `[:10]` to only show the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print("------")
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

In [23]:
table = doc.select_one('tbody')
table_rows = table.find_all('tr')

for row in table_rows:
    print(row.get_text()) 

Name and LocationOrderBasis for Order
 NGUYEN, THU THI  City: HOUSTON County: HARRIS Zip Code: 77031 License #: 816854Complaint # COS20230009980 Date: 10/23/2023Respondents Thu Thi Nguyen and Tien Minh Vo are assessed an administrative penalty in the amount of $1,350. Respondents failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondents failed to discard single use implements after each use; Respondents failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions.
 NGUYEN, TUONG  City: FORT WORTH County: TARRANT Zip Code: 76120 License #: 742271Complaint # COS20230001960 Date

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll probably get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – like `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [40]:
for row in table_rows[6]:
    print(row)

 
<td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:22%;"><span class="results_text">NGUYEN, TAMMY </span><br/> <span class="default_text">City:</span> <span class="results_text">TOMBALL</span><br/> <span class="default_text">County:</span> <span class="results_text">HARRIS</span><br/> <span class="default_text">Zip Code:</span> <span class="results_text">77375</span><br/><br/><span class="results_text">NGUYEN, THANG </span><br/> <span class="default_text">City:</span> <span class="results_text">TOMBALL</span><br/> <span class="default_text">County:</span> <span class="results_text">HARRIS</span><br/> <span class="default_text">Zip Code:</span> <span class="results_text">77375</span><br/><br/><br/><span class="default_text"> License #:</span> <span class="results_text">789659</span><br/><br/><span class="default_text">Complaint #</span> <span class="results_text">COS20220013443</span></td>
 
<td style="padding:4px; text-align:left; font-s

In [43]:
for row in table_rows:
    td_tag = row.find('td')
    if td_tag:
        results_spans = td_tag.find_all('span', class_='results_text')
        for span in results_spans:
            text = span.get_text().strip()
            if 'NGUYEN,' in text:
                print(text)

NGUYEN, THU THI
NGUYEN, TUONG
NGUYEN, HUONG T
NGUYEN, STEVEN QUOC
NGUYEN, HUONG THI NGOC
NGUYEN, TAMMY
NGUYEN, THANG
NGUYEN, TOAN DUC
NGUYEN, NHAN TRONG
NGUYEN, TAM PHUONG
NGUYEN, NHUNG NGOC THI
NGUYEN, ANH NGOC
NGUYEN, TIFFANY
NGUYEN, HUYEN THI
NGUYEN, DANIEL
NGUYEN, BANG
NGUYEN, HONG N
NGUYEN, THANH
NGUYEN, TUYET
NGUYEN, TUYET MINH
NGUYEN, JOHN THANH NHAN
NGUYEN, THE THI
NGUYEN, THAO HONG
NGUYEN, NGOC THI
NGUYEN, TUYEN THANH
NGUYEN, MINH THU
NGUYEN, LOAN THI NGOC
NGUYEN, JACK
NGUYEN, HOANG DRAGON
NGUYEN, KELLY
NGUYEN, NIEN
NGUYEN, AJ
NGUYEN, CHARLIE
NGUYEN, OANH THI THUY
NGUYEN, LOC
NGUYEN, THAI VAN
NGUYEN, THUY THU THI
NGUYEN, DUC VAN
NGUYEN, HUNG THANH
NGUYEN, MIKE NHAN
NGUYEN, SAMANTHA T
NGUYEN, THOMAS
NGUYEN, HIEU VAN
NGUYEN, MYTU THI
NGUYEN, NGOC
NGUYEN, BINH PHUONG THI
NGUYEN, QUI NGOC
NGUYEN, TRINH
NGUYEN, MUOI
NGUYEN, CHARLIE
NGUYEN, TONY VAN
NGUYEN, NGOC THAO DANG
NGUYEN, KELLY PHUONG N
NGUYEN, QUOC BAO
NGUYEN, DANIEL THAO
NGUYEN, THANH THU
NGUYEN, CINDY
NGUYEN, TRANG N M
NG

## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: Or I guess you could just skip the one with the problem using try/except...*

In [31]:
for row in table_rows:
    print("***")
    td_tag = row.find('td')
    if td_tag:
        tds = row.find_all('td')
        complaint_td = tds[2] 
        print(complaint_td.get_text().strip())
    else:
        print("No complaint found in this row")

***
No complaint found in this row
***
Respondents failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondents failed to discard single use implements after each use; Respondents failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions.
***
Respondents employed an individual as an operator who had not obtained a cosmetology license; Respondents failed to follow whirlpool foot spas cleaning and sanitization procedures as required, the Department is charging 2 violations; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved steril

## Loop through each result, printing the complaint number

Output should look similar to this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [60]:
for row in table_rows:
    td_tag = row.find('td')
    if td_tag:
        results = td_tag.find_all('span', class_='results_text')
        for result in results:
            com_numbers = re.findall(r'COS\d{11}', result.get_text())
            for com_number in com_numbers:
                print(com_number)
    else:
        print("No complaint number found")

No complaint number found
COS20230009980
COS20230001960
COS20220017901
COS20220018135
COS20220016107
COS20220013443
COS20220015714
COS20220012646
COS20220012645
COS20230010084
COS20230010404
COS20220015853
COS20220008852
COS20230009558
COS20230010492
COS20230000043
COS20220006122
COS20220009835
COS20210007272
COS20210012244
COS20210016437
COS20220007001
COS20220009623
COS20220014041
COS20220016162
COS20210009264
COS20210012950
COS20210013902
COS20220005426
COS20210012469
COS20220004591
COS20220005795
COS20230000013
COS20220000293
COS20210012920
COS20220006428
COS20220011063
COS20230002643
COS20220012452
COS20220009944
COS20210014192
COS20220016639
COS20210013960
COS20210015021
COS20210015551
COS20210016400
COS20220006797
COS20210009267
COS20220004501
COS20210012742
COS20220004954
COS20210004823
COS20220002667
COS20220000853
COS20200013887
COS20210010437
COS20210014779
COS20200015190
COS20200008154
COS20200007790
COS20210010525
COS20210015763
COS20210011311
COS20210003963
COS20190017129

## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> _**Tip:** Depending on how you do this, you might want to figure out how to search for "next siblings" or "following siblings"_

In [73]:
complaint_list = []
for row in table_rows:
    td_tag = row.find('td')
    names = []
    if td_tag:
        results_spans = td_tag.find_all('span', class_='results_text')
        tds = row.find_all('td')
        td_text = td_tag.get_text()
        complaint_no_match = re.search(r'COS\d{11}', td_text)
        if complaint_no_match:
            complaint_no = complaint_no_match.group(0) 
        else:
            None
        for span in results_spans:
            text = span.get_text().strip()
            if 'NGUYEN,' in text:
                names.append(text)
                complaint = {
                'Name': names,
                'City': td_tag.find_all('span', class_='results_text')[1].get_text().strip(),
                'County': td_tag.find_all('span', class_='results_text')[2].get_text().strip(),
                'Zip': td_tag.find_all('span', class_='results_text')[3].get_text().strip(),
                'Complaint_No':complaint_no,
                'License_No': td_tag.find_all('span', class_='results_text')[-2].get_text().strip(),
                'Complaint': tds[2].get_text().strip()
                 }
                complaint_list.append(complaint)

In [74]:
complaint_list

[{'Name': ['NGUYEN, THU THI'],
  'City': 'HOUSTON',
  'County': 'HARRIS',
  'Zip': '77031',
  'Complaint_No': 'COS20230009980',
  'License_No': '816854',
  'Complaint': "Respondents failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondents failed to discard single use implements after each use; Respondents failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions."},
 {'Name': ['NGUYEN, TUONG'],
  'City': 'FORT WORTH',
  'County': 'TARRANT',
  'Zip': '76120',
  'Complaint_No': 'COS20230001960',
  'License_No': '742271',
  'Complaint': "Respondents employed an individual as

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [75]:
import pandas as pd

In [76]:
df = pd.DataFrame(complaint_list)

In [78]:
df.to_csv('output.csv')

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [79]:
pd.read_csv('output.csv').head(5)

,Unnamed: 0,Name,City,County,Zip,Complaint_No,License_No,Complaint
0,0,"['NGUYEN, THU THI']",HOUSTON,HARRIS,77031,COS20230009980,816854,"Respondents failed to clean, disinfect, and st..."
1,1,"['NGUYEN, TUONG']",FORT WORTH,TARRANT,76120,COS20230001960,742271,Respondents employed an individual as an opera...
2,2,"['NGUYEN, HUONG T']",SAN ANTONIO,BEXAR,78259,COS20220017901,775126,Respondent failed to clean and disinfect facia...
3,3,"['NGUYEN, STEVEN QUOC']",SAN ANGELO,TOM GREEN,76904,COS20220018135,"748707, 1458935",Respondent failed to cooperate with the inspec...
4,4,"['NGUYEN, HUONG THI NGOC']",MANSFIELD,TARRANT,76063,COS20220016107,829081,"Respondent failed to clean, disinfect, and ste..."
